This notebook preps data for Tableau maps and other visualizations

Author: Timur Mukhtarov

In [1]:
# import libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import xlrd
import shapely
import sys
from fiona.crs import from_epsg

# Remove the limit for number of columns shown in a dataframe
pd.options.display.max_columns = None 

In [2]:
# Load in data

# load in the shapefile for New York City Census Tracts
shape_file = gpd.read_file('/Users/tm/Dropbox/NYU/Capstone/Capstone Data/Other/2010 New York City Census Tract Boundaries/nyu_2451_34513/nyu_2451_34513.shp')
# shape_file.head()

# load in the shapefile created in the merging_shapefile_and_census_broadband_data notebook
ct_shp = gpd.read_file('/Users/tm/Dropbox/NYU/Capstone/Git/EDA_data_prep_timur/Broadband/data/ctshp_bb_1/ctshp_bb_1.shp') 

# load in data for EBT calls made from LinkNYC stations, aggregated by a Link
ebt_data = pd.read_excel('/Users/tm/Dropbox/NYU/Capstone/Capstone Data/Link EBT Call Data 041219.xlsx').sort_values('Site ID')
# ebt_data.head()

# load in data for impressions numbers for each Link. Impressions are a marketing term associated with how many
# people see an ad. Intersection uses a third-party provider for these data.
impressions_data = pd.read_excel('/Users/tm/Dropbox/NYU/Capstone/Capstone Data/data/LinkNYC Impression Multipliers 041219.xlsx').sort_values('Site ID')
# impressions_data.head()

# load in LinkNYC usage for July 2018
link_data_july_2018 = pd.read_csv('/Users/tm/Dropbox/NYU/Capstone/Capstone Data/data/July 2018 Monthly LinkNYC Data - Per-AP.csv').sort_values('AP Device Name')
# link_data_july_2018.head()

link_locations = pd.read_csv('/Users/tm/Dropbox/NYU/Capstone/Capstone Data/data/LinkNYC_Locations.csv')
policy_map_bb = pd.read_csv('/Users/tm/Dropbox/NYU/Capstone/Git/EDA_data_prep_timur/Broadband/data/policymap_broadband_nyc.csv')
policy_map_snap = pd.read_csv('/Users/tm/Dropbox/NYU/Capstone/Git/EDA_data_prep_timur/Broadband/data/policymap_est_percent_of_all_families_rec_SNAP_benefits.csv')

census_data = pd.read_csv('/Users/tm/Desktop/Census/Census_data.csv')

In [8]:
# Keep only relevant columns from Link locations 
link_locations = link_locations[['Latitude', 'Longitude', 'Link Site ID', 
                                'Link Installation (A)', 'Neighborhood Tabulation Area (NTA)',
                                'Census Tract (CT)', 'Location']]

# Rename ID column to merge tables on Link ID later
link_locations = link_locations.rename(columns={'Link Site ID':'Site ID'})
link_data_july_2018 = link_data_july_2018.rename(columns={'AP Device Name':'Site ID'})
# link_locations.head()

In [12]:
# Merge tables
merged = link_locations.merge(ebt_data, on='Site ID', how='left')
merged = merged.merge(link_data_july_2018, on='Site ID', how='left')
merged = merged.merge(impressions_data, on='Site ID', how='left')
merged.head()

,Latitude_x,Longitude_x,Site ID,Link Installation (A),Neighborhood Tabulation Area (NTA),Census Tract (CT),Location,﻿Activation Date,Days Since Activation,Borough,Community Board,Latitude_y,Longitude_y,Street Address,Zip Code,EBT Total Calls,EBT Calls Per Day
0,40.767358,-73.982564,mn-04-144139,07/20/2016,Clinton,1013900,"(40.76735759, -73.98256354)",2016-08-03,840.0,Manhattan,104.0,40.767358,-73.982564,989 8 AVENUE,10019.0,213.0,0.253571
1,40.747055,-73.981061,mn-06-133555,03/03/2018,Murray Hill-Kips Bay,1007200,"(40.74705459, -73.98106107)",2018-05-17,319.0,Manhattan,106.0,40.747055,-73.981061,99 EAST 34 STREET,10016.0,73.0,0.228840
2,40.801949,-73.949259,mn-10-138528,12/21/2016,Central Harlem South,1019000,"(40.80194888, -73.9492591)",2017-02-06,784.0,Manhattan,110.0,40.801949,-73.949259,99 WEST 116 STREET,10026.0,771.0,0.983418
3,40.726695,-73.853929,qu-06-145847,05/20/2017,Forest Hills,4071306,"(40.726695, -73.853929)",2017-07-12,628.0,Queens,406.0,40.726695,-73.853929,99-40 67 AVENUE,11374.0,10.0,0.015924
4,40.761604,-73.966388,mn-08-121734,02/27/2016,Lenox Hill-Roosevelt Island,1011000,"(40.76160412, -73.9663883)",2016-03-29,840.0,Manhattan,108.0,40.761604,-73.966388,991 3 AVENUE,10022.0,23.0,0.027381


In [15]:
# delete some of the repeating columns 
merged = merged.drop(columns=['Latitude_y','Longitude_y','Street Address_y'])

# Check how many rows for different columns
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1868 entries, 0 to 1867
Data columns (total 38 columns):
Latitude_x                               1868 non-null float64
Longitude_x                              1868 non-null float64
Site ID                                  1868 non-null object
Link Installation (A)                    1868 non-null object
Neighborhood Tabulation Area (NTA)       1868 non-null object
Census Tract (CT)                        1868 non-null int64
Location                                 1868 non-null object
﻿Activation Date                         1772 non-null datetime64[ns]
Days Since Activation                    1772 non-null float64
Borough                                  1772 non-null object
Community Board                          1772 non-null float64
Latitude_y                               1772 non-null float64
Longitude_y                              1772 non-null float64
Street Address_x                         1772 non-null object
Zip Code    

In [18]:
# Check if all columns merged correctly
merged.columns

Index(['Latitude_x', 'Longitude_x', 'Site ID', 'Link Installation (A)',
       'Neighborhood Tabulation Area (NTA)', 'Census Tract (CT)', 'Location',
       '﻿Activation Date', 'Days Since Activation', 'Borough',
       'Community Board', 'Street Address_x', 'Zip Code', 'EBT Total Calls',
       'EBT Calls Per Day', 'Boro', 'CB', 'PPT ID', 'Latitude', 'Longitude',
       'SSID(s)', 'Uptime %', 'Number of Unique Clients', 'Number of Sessions',
       'Minutes Used', 'MB Transferred (upload)', 'MB Transferred (download)',
       'MB Transferred (total)', 'Daily Avg - Number of Unique Clients',
       'Daily Avg - Number of Sessions', 'Daily Avg - Minutes Used',
       'Daily Avg - MB Transferred (upload)',
       'Daily Avg - MB Transferred (download)',
       'Daily Avg - MB Transferred (total)', 'Impressions per 15 seconds'],
      dtype='object')

In [19]:
# Check if all columns merged correctly
merged.head()

,Latitude_x,Longitude_x,Site ID,Link Installation (A),Neighborhood Tabulation Area (NTA),Census Tract (CT),Location,﻿Activation Date,Days Since Activation,Borough,Community Board,Street Address_x,Zip Code,EBT Total Calls,EBT Calls Per Day,Boro,CB,PPT ID,Latitude,Longitude,SSID(s),Uptime %,Number of Unique Clients,Number of Sessions,Minutes Used,MB Transferred (upload),MB Transferred (download),MB Transferred (total),Daily Avg - Number of Unique Clients,Daily Avg - Number of Sessions,Daily Avg - Minutes Used,Daily Avg - MB Transferred (upload),Daily Avg - MB Transferred (download),Daily Avg - MB Transferred (total),Impressions per 15 seconds
0,40.767358,-73.982564,mn-04-144139,07/20/2016,Clinton,1013900,"(40.76735759, -73.98256354)",2016-08-03,840.0,Manhattan,104.0,989 8 AVENUE,10019.0,213.0,0.253571,mn,4.0,144139.0,40.767358,-73.982564,"LinkNYC Free Wi-Fi, LinkNYC Private, SONYC IOT",99.60%,"45,094","93,849","266,739","39,192","186,332","225,524","1,455","3,027","8,604","1,264","6,011","7,275",14.789137
1,40.747055,-73.981061,mn-06-133555,03/03/2018,Murray Hill-Kips Bay,1007200,"(40.74705459, -73.98106107)",2018-05-17,319.0,Manhattan,106.0,99 EAST 34 STREET,10016.0,73.0,0.228840,mn,6.0,133555.0,40.747055,-73.981061,"LinkNYC Free Wi-Fi, LinkNYC Private, SONYC IOT",100.00%,"23,130","47,019","129,417","23,286","161,478","184,763",746,"1,517","4,175",751,"5,209","5,960",9.313095
2,40.801949,-73.949259,mn-10-138528,12/21/2016,Central Harlem South,1019000,"(40.80194888, -73.9492591)",2017-02-06,784.0,Manhattan,110.0,99 WEST 116 STREET,10026.0,771.0,0.983418,mn,10.0,138528.0,40.801949,-73.949259,"LinkNYC Free Wi-Fi, LinkNYC Private, SONYC IOT",99.46%,"35,201","170,942","672,007","131,066","899,669","1,030,736","1,136","5,514","21,678","4,228","29,022","33,250",5.126637
3,40.726695,-73.853929,qu-06-145847,05/20/2017,Forest Hills,4071306,"(40.726695, -73.853929)",2017-07-12,628.0,Queens,406.0,99-40 67 AVENUE,11374.0,10.0,0.015924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.387202
4,40.761604,-73.966388,mn-08-121734,02/27/2016,Lenox Hill-Roosevelt Island,1011000,"(40.76160412, -73.9663883)",2016-03-29,840.0,Manhattan,108.0,991 3 AVENUE,10022.0,23.0,0.027381,mn,8.0,121734.0,40.761604,-73.966388,"LinkNYC Free Wi-Fi, LinkNYC Private, SONYC IOT",99.87%,"38,176","96,200","199,604","18,906","112,764","131,670","1,231","3,103","6,439",610,"3,638","4,247",14.721875


In [22]:
# Change dtypes for some of the columns of interest

column_list = ['Number of Unique Clients', 'Number of Sessions',
        'Minutes Used', 'MB Transferred (upload)', 'MB Transferred (download)',
        'MB Transferred (total)', 'Daily Avg - Number of Unique Clients',
        'Daily Avg - Number of Sessions', 'Daily Avg - Minutes Used',
        'Daily Avg - MB Transferred (upload)',
        'Daily Avg - MB Transferred (download)',
        'Daily Avg - MB Transferred (total)']

for i in column_list:
    merged[i] = merged[i].str.replace(",","").astype(float)
    

In [33]:
# Create LinkNYC station Geometry (Point) from Coordinates (Lat & Lon)

# combine lat and lon to one column
merged['lonlat']=list(zip(merged.Longitude_x,merged.Latitude_x))
# Create Point Geometry for based on lonlat column
merged['geometry']=merged[['lonlat']].applymap(lambda x:shapely.geometry.Point(x))

# check if the new column was created correctly
merged.head(1)

In [36]:
# convert coordinates for pumashp and linkNYC in prep for sjoin (spatial join using GeoPandas)

# We need to do this since spatial joins are done in cartesian geometry and 
#the only coordinate systems suitable to spatial joins are flat coordinate plane systems. 
#In the NYC area, such system is 2263

merged.crs = from_epsg(2263) 
ct_shp.crs = from_epsg(2263)

/anaconda3/envs/bigdata/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  import sys


In [38]:
# Count number of LinkNYC stations in each census tract
link_ct_shp = gpd.sjoin(merged, ct_shp, how = 'left', op='intersects').groupby('Census Tra').count()
link_ct_shp = link_ct_shp['Site ID'].to_frame('number_link_stations')
link_ct_shp.head()

ValueError: 'left_df' should be GeoDataFrame, got <class 'pandas.core.frame.DataFrame'>

In [39]:
# Count LinkNYC usage data per census tract

# keep relevant columns
mask = ['EBT Total Calls', 'EBT Calls Per Day', 'Number of Unique Clients', 'Number of Sessions',
       'Minutes Used', 'MB Transferred (upload)', 'MB Transferred (download)',
       'MB Transferred (total)', 'Daily Avg - Number of Unique Clients',
       'Daily Avg - Number of Sessions', 'Daily Avg - Minutes Used',
       'Daily Avg - MB Transferred (upload)',
       'Daily Avg - MB Transferred (download)',
       'Daily Avg - MB Transferred (total)', 'Impressions per 15 seconds']

link_sum_shp = gpd.sjoin(merged, ct_shp, how = 'left', op='intersects').groupby('Census Tra').sum()
link_sum_shp = link_sum_shp[mask]

ValueError: 'left_df' should be GeoDataFrame, got <class 'pandas.core.frame.DataFrame'>

In [ ]:
# Merge
ct_shp = ct_shp.merge(link_ct_shp, on='Census Tra', how='outer')
ct_shp = ct_shp.merge(link_sum_shp, on='Census Tra', how='outer')
ct_shp.head(20)

In [ ]:
# Get a clean Census Tract (without ', NY' in the end)
ct_shp['Census Tract'] = ct_shp['Census Tra'].apply(str)
ct_shp['census_tract_clean'] = ct_shp['Census Tract'].str[0:11]
ct_shp.head()

In [ ]:
# Rename columns for merging 
policy_map_bb = policy_map_bb[['Formatted FIPS', '% of households w/o broadband']].rename(columns={'Formatted FIPS':'census_tract_clean', '% of households w/o broadband':'HouseholdsPercentWithoutBroadband'})
policy_map_snap = policy_map_snap[['Formatted FIPS', '% on SNAP ']].rename(columns={'Formatted FIPS':'census_tract_clean', '% on SNAP ': 'HousehouldsPercentOnSNAP'})
census_data = census_data.rename(columns={'Id2':'census_tract_clean'})


In [ ]:
# Merge Census data and the shapefile 
census_data = census_data.merge(policy_map_snap, on='census_tract_clean', how='left')
census_data = census_data.merge(policy_map_bb, on='census_tract_clean', how='left')
census_data['census_tract_clean'] = census_data['census_tract_clean'].astype('str')
ct_shp = ct_shp.merge(census_data, on='census_tract_clean', how='left')

In [ ]:
# Check columns in the resulting shapefile
ct_shp.columns

In [ ]:
# Frop unnecessary columns
ct_shp = ct_shp.drop(columns=['boro_code', 'boro_ct201', 
             'cdeligibil', 'ct2010', 
             'ctlabel', 'ntacode', 
             'Census Tra','census_tra',
             'census_t_1','borough',
             'borough_ce'])

ct_shp = ct_shp.drop(columns=['% of house'])

In [ ]:
# See the resulted dataframe
ct_shp.head()

In [ ]:
# See the list of the column names
ct_shp.columns

In [ ]:
# Rename columns for easier export and in preparation for work in Tableau

ct_shp = ct_shp.rename(columns={'boro_name':'Borough', 
                        'number_link_stations':'NumberOfLinkStations', 
                        'EBT Total Calls':'EBTTotalCalls',
                        'EBT Calls Per Day':'EBTCallsPerDay', 
                        'Number of Unique Clients':'NumberOfUniqueClients',
                        'Number of Sessions':'NumberOfSessions',
                        'Minutes Used':'MinutesUsed', 
                        'MB Transferred (upload)':'MBTransferredUpload', 
                        'MB Transferred (download)':'MBTransferredDownload',
                        'MB Transferred (total)':'MBTransferredTotal', 
                        'Daily Avg - Number of Unique Clients':'DailyAvgNumberOfUniqueClients',
                        'Daily Avg - Number of Sessions':'DailyAvgNumberOfSessions',
                        'Daily Avg - Minutes Used':'DailyAvgMinutesUsed',
                        'Daily Avg - MB Transferred (upload)':'DailyAvgMBTransferredUpload',
                        'Daily Avg - MB Transferred (download)':'DailyAvgMBTransferredDownload',
                        'Daily Avg - MB Transferred (total)':'DailyAvgMBTransferredTotal', 
                        'Impressions per 15 seconds':'ImpressionsPer15Seconds',
                        'Census Tract':'CensusTract', 
                        'census_tract_clean':'CensusTractClean', 
                        'total_households':'HouseholdTotal', 
                        '%_household_income_less_10_000':'HouseholdPercentIncomeLess10000',
                        '%_household_income_10_000_14_999':'HouseholdPercentIncomebet10000and14999', 
                        '%_income_15_000_24_999':'HouseholdPercentIncomebet15000and24999',
                        '%_household_income_25_000_34_999':'HouseholdPercentIncomebet25000and34999', 
                        '%_household_income_35_000_49_999':'HouseholdPercentIncomebet35000and49999',
                        '%_household_income_50_000_74_999':'HouseholdPercentIncomebet50000and74999', 
                        '%_household_income_75_000_99_999':'HouseholdPercentIncomebet75000and99999',
                        '%_household_income_100_000_149_999':'HouseholdPercentIncomebet100000and149999',
                        '%_household_income_150_000_199_999':'HouseholdPercentIncomebet150000and199999', 
                        '%_household_income_200_000_plus':'HouseholdPercentIncome200000plus',
                        'household_income_median':'HouseholdMedianIncome', 
                        'household_income_mean':'HouseholdMeanIncome',
                        'education_total_population':'EducationTotalPopulation', 
                        'education_total_pop_high_school':'EducationTotalPopHighSchool',
                        'education_total_pop_ged':'EducationTotalPopGED', 
                        'education_total_pop_associates':'EducationTotalPopAssociates',
                        'education_total_pop_bachelors':'EducationTotalPopBachelors', 
                        'education_total_pop_masters':'EducationTotalPopMasters',
                        'education_total_pop_prof_degree':'EducationTotalProfDegree', 
                        'education_total_pop_doctorate':'EducationTotalPopDoctorate',
                        'race_total_population':'RaceTotalPopulation',
                        'race_total_black_alone':'RaceTotalWhite',
                        'race_total_black_alone.1':'RaceTotalBlack', 
                        'race_total_am_indian_alaskan_native_alone':'RaceTotalAmIndianOrAlaskanNative',
                        'race_total_asian_alone':'RaceTotalAsian',
                        'race_total_native_hawaiian_other_pacific_islander_alone':'RaceTotalNativeHawaiianOrOtherPacificIslander',
                        'race_total_some_other_race_alone':'RaceTotalSomeOtherRace', 
                        'race_total_two_or_more_races':'RaceTotalTwoOrMoreRaces',
                        'occupancy_total_population':'OccupancyTotalPopulation', 
                        'occupancy_total_owner_occupied':'OccupancyTotalOwnerOccupied',
                        'occupancy_total_renter_occupied':'OccupancyTotalRenterOccupied', 
                        'insurance_percent':'PercentofPopulationWithoutInsurance',
                        'all_below_poverty_level_percent':'PercentofPopBelowPovertyLevel', 
                        'hisporigin_total':'HisporiginTotal',
                        'hisporigin_not_hisp_or_latino':'HisporiginNotHispOrLatino', 
                        'hisporigin_hisp_or_latino':'HisporiginHispOrLatino',
                        'median_gross_rent_as_%_of_household_income':'MedianGrossRentAsPercentOfHouseholdIncome'})

In [ ]:
# Output to a shapefile
# Now the shapefile has both LinkNYC and Census data by NYC Census Tracts
ct_shp.to_file('ct_with_linknyc_data_and_census_data.shp')